# MovieTinder

file to do the cleaning, while Jupiter notebook has better visualization for tables.

In [3]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import ast

df = pd.read_csv('data/movies_metadata_org.csv')
print("Original shape: ", df.shape)


# drop the columns which have a missing value percentage of above 55 
toBeDropped = df.columns[df.isnull().mean() > 0.55] 
df = df.drop(columns= toBeDropped)
print("Shape after dropping NaN columns: ", df.shape, " Columns deleted: ", len(toBeDropped))

# from this we can see that most rows are complete (without missing values) while only 10 rows have less than 21 finite values in their row with 21
# being the total amount of colums, therefore we do not drop any rows found by this. 
# print(len(df.dropna(thresh=20)))

title = df['title']
df = df.drop(columns = ['original_title', 'title', 'status', 'video', 'poster_path', 'production_countries','spoken_languages'])
df.insert(0, 'Title', title) #move title to the front
print("Shape after dropping unwanted columns: ", df.shape)

df = df[df['production_companies'].notna()]

#  rows which have a boolean type value for their production companies, yes i did this manualy and yes it could have been automated
df = df.drop(index = 29503)
df = df.drop(index = 19730)
df = df.drop(index = 35587)


# clean the genres and production companies columns

for index, row in df.iterrows():
    res = ast.literal_eval(row['genres']) 
    res2 = ast.literal_eval(row['production_companies'])
    genres = []
    companies = []
    
    
    for genre in res:
        genres.append(genre['name'])
    
    for comp in res2:
        companies.append(comp['name'])
    
    if len(companies) == 0:
        df.at[index,'production_companies'] = np.nan
    else: 
        df.at[index,'production_companies'] = companies
    
    if len(genres) == 0:
        df.at[index,'genres'] = np.nan
    else: 
        df.at[index, 'genres'] = genres

# drop na in genre and na in imdb_id
df = df[df['genres'].notna()]
df = df[df['imdb_id'].notna()]


Original shape:  (45466, 24)
Shape after dropping NaN columns:  (45466, 21)  Columns deleted:  3
Shape after dropping unwanted columns:  (45466, 15)


In [6]:
print(df.vote_count.mean())
print(df.vote_count.max())

# add weightedRating column in the dataset to take into account the user votes and the rating, just a simple multiplication 
df['weightedRating'] = df['vote_count'] * df['vote_average']
print(len(df[df.weightedRating > 1000]))

# plt.xlim([1000,6000])
# plt.hist(df.weightedRating, bins = 100)
# plt.show()

115.9984188252802
14075.0
4558


In [7]:
df.to_csv('data/movieData.csv', sep=',', encoding='utf-8', index = False)

In [8]:
# check if everything went fine
df = pd.read_csv('data/movieData.csv',error_bad_lines=False)
df.head()

Title  adult    budget  \
0                    Toy Story  False  30000000   
1                      Jumanji  False  65000000   
2             Grumpier Old Men  False         0   
3            Waiting to Exhale  False  16000000   
4  Father of the Bride Part II  False         0   

                               genres     id    imdb_id original_language  \
0   ['Animation', 'Comedy', 'Family']    862  tt0114709                en   
1  ['Adventure', 'Fantasy', 'Family']   8844  tt0113497                en   
2               ['Romance', 'Comedy']  15602  tt0113228                en   
3      ['Comedy', 'Drama', 'Romance']  31357  tt0114885                en   
4                          ['Comedy']  11862  tt0113041                en   

                                            overview  popularity  \
0  Led by Woody, Andy's toys live happily in his ...   21.946943   
1  When siblings Judy and Peter discover an encha...   17.015539   
2  A family wedding reignites the ancient feud be...   11.712900   
3  Cheated on, mistreated and stepped on, the wom...    3.859495   
4  Just when George Banks has recovered from his ...    8.387519   

                                production_companies release_date  \
0                        ['Pixar Animation Studios']   1995-10-30   
1  ['TriStar Pictures', 'Teitler Film', 'Intersco...   1995-12-15   
2                 ['Warner Bros.', 'Lancaster Gate']   1995-12-22   
3         ['Twentieth Century Fox Film Corporation']   1995-12-22   
4   ['Sandollar Productions', 'Touchstone Pictures']   1995-02-10   

       revenue  runtime  vote_average  vote_count  weightedRating  
0  373554033.0     81.0           7.7      5415.0         41695.5  
1  262797249.0    104.0           6.9      2413.0         16649.7  
2          0.0    101.0           6.5        92.0           598.0  
3   81452156.0    127.0           6.1        34.0           207.4  
4   76578911.0    106.0           5.7       173.0           986.1